# WIE3007 Group Assignmet - Dataset Generation

## Generating Dataset

### Importing Libraries

In [ ]:
import io
import pandas as pd
import asyncio
from openai import AsyncOpenAI
import getpass

### Setting the API Key

In [29]:
api_key = getpass.getpass("OpenAI API Key: ")

OpenAI API Key: ··········


### Prompting and Calling API

In [30]:
BATCH_SIZE = 50
TOTAL_BATCHES = 40

PROMPT_TEMPLATE = """
You are a data generator. Produce realistic customer churn data of {batch_size} for a subscription-based digital service.

REQUIREMENTS:
1. Output ONLY CSV, no explanations, no markdown.
2. Include this header EXACTLY (in this order):
customer_id,age,gender,subscription_type,tenure_months,monthly_spend,support_tickets,avg_session_time,customer_feedback,churn
3. Each row must have EXACTLY 10 columns, comma-separated.
4. Enclose textual feedback (customer_feedback) in double quotes " " to handle commas inside text.
5. Generate realistic numerical and categorical data:
   - customer_id: random 8-digit integer, with no discrnible patterns
   - age: 18–65
   - gender: M or F
   - subscription_type: Basic / Standard / Premium
   - tenure_months: 1–48
   - monthly_spend: RM15–RM200 (float, 2 decimals)
   - support_tickets: 0–20
   - avg_session_time: 10–120 minutes/day (float, 1 decimal)
   - customer_feedback: short human-like sentence (max 12 words)
   - churn: 0 = stayed, 1 = left
6. Apply business logic:
   - New users (tenure < 3 months) are more likely to churn.
   - High support_tickets (>5) increase churn probability.
   - Premium users are less likely to churn.
   - Negative feedback sentences → churn more likely.
   - Monthly spend and usage affect churn probability realistically.
7. Ensure the churn label matches the business logic.
8. Generate {rows} rows per request.
9. Avoid extra blank lines or extra text. Strictly return CSV only.
"""

In [31]:
client = AsyncOpenAI(api_key=api_key)

async def fetch_batch():
    # Calculate the end ID for the prompt
    # end_id = start_id + BATCH_SIZE - 1

    # Format the template with the dynamic values
    prompt = PROMPT_TEMPLATE.format(batch_size=BATCH_SIZE, rows=BATCH_SIZE).strip()

    response = await client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [32]:
async def main():
    tasks = []

    # Logic to calculate the unique starting ID for each batch
    for i in range(TOTAL_BATCHES):
        # start_id = 1 + (i * BATCH_SIZE)
        tasks.append(fetch_batch())

    # Gather results as they come in
    results = await asyncio.gather(*tasks)
    return results

In [33]:
results = await main()

## Data Cleaning

### Combining into One DataFrame

In [34]:
combined_data = []

# Keep the first batch (includes the header)
combined_data.append(results[0])

# Loop through the remaining batches and discard their headers
for csv_string in results[1:]:
    # Split by newline, discard the first line (lines[1:]), and join back
    lines = csv_string.strip().split('\n')
    combined_data.append('\n'.join(lines[1:]))

# Join all chunks into the final string
final_csv_string = '\n'.join(combined_data)

In [35]:
try:
    # Split the string into header and data
    lines = final_csv_string.split('\n')
    header = lines[0]

    fixed_csv_string = '\n'.join(lines)

    df = pd.read_csv(
        io.StringIO(fixed_csv_string),
        on_bad_lines='skip',
        sep=',',
        quotechar='"'
    )
except Exception as e:
    print(f"An error occurred: {e}")

### Dataset after Combining to DataFrame

In [36]:
df

,customer_id,age,gender,subscription_type,tenure_months,monthly_spend,support_tickets,avg_session_time,customer_feedback,churn
0,85714623,34.0,M,Standard,12.0,45.50,3.0,30.5,Great service but needs more features.,0.0
1,29547381,28.0,F,Premium,24.0,150.00,1.0,75.2,Love the content offerings!,0.0
2,33416294,45.0,M,Standard,9.0,50.00,6.0,35.0,"Not bad, but slow response times.",1.0
3,19486372,55.0,F,Premium,36.0,180.00,0.0,90.0,Best service I've used so far!,0.0
4,87126314,22.0,M,Standard,2.0,30.00,8.0,20.0,I might consider leaving soon.,1.0
...,...,...,...,...,...,...,...,...,...,...
1673,75158072,49.0,F,Premium,42.0,RM175.00,0.0,95.0,Absolutely essential for me,0.0
1674,85473219,55.0,M,Standard,20.0,RM89.00,1.0,48.0,"Good value, content needs updates",0.0
1675,41386029,33.0,F,Basic,1.0,RM15.00,9.0,10.5,"Terrible service, I'm out",1.0
1676,59420178,45.0,M,Premium,39.0,RM180.00,0.0,110.0,"Always reliable, will renew",0.0


### Handling Missing Values and Wrong Data Types

#### Installing and Importing Libraries

In [64]:
!pip install langid langedetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941171 sha256=a2d970fcb6172ff9e24a3f01bbb3cfb9cbcf635608c47a7413d2f78046deadec
  Stored in directory: /root/.cache/pip/wheels/3c/bc/9d/266e27289b9019680d65d9b608c37bff1eff565b001c977ec5
Successfully built langid


In [ ]:
from langdetect import detect, DetectorFactory
import langid

#### Checking Current Data Types

In [55]:
df.dtypes

,0
customer_id,object
age,float64
gender,object
subscription_type,object
tenure_months,float64
monthly_spend,object
support_tickets,float64
avg_session_time,object
customer_feedback,object
churn,float64


Wrong Data Type for customer_id, monthly_spend, support_tickets, avg_session_time

#### Deleting Rows with Wrong Data Type

In [56]:
numeric_cols = [
    "customer_id",
    "age",
    "tenure_months",
    "monthly_spend",
    "support_tickets",
    "avg_session_time",
    "churn"
]

In [57]:
df_clean = df.copy()

error_mask = pd.Series(False, index=df_clean.index)

for col in numeric_cols:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    error_mask = error_mask | df_clean[col].isna()

print(f"Number of rows with invalid numeric values: {error_mask.sum()}")

Number of rows with invalid numeric values: 305


In [58]:
df_clean = df_clean[~error_mask].reset_index(drop=True)
print(f"Cleaned dataset now has {len(df_clean)} rows.")

Cleaned dataset now has 1373 rows.


#### Checking the Data Types after Removal

In [59]:
df_clean.dtypes

,0
customer_id,float64
age,float64
gender,object
subscription_type,object
tenure_months,float64
monthly_spend,float64
support_tickets,float64
avg_session_time,float64
customer_feedback,object
churn,float64


#### Verifying the contents of Text Data

In [60]:
categorical_cols = [
    "gender",
    "subscription_type",
    "customer_feedback"
]

In [61]:
for col in categorical_cols:
    print(df_clean[col].value_counts())

gender
M    690
F    683
Name: count, dtype: int64
subscription_type
Standard    467
Premium     456
Basic       449
国产            1
Name: count, dtype: int64
customer_feedback
Best investment I've made!                7
Too basic for my needs.                   6
Excellent service, highly recommend!      5
Not enough features for the price.        5
Absolutely love this service!             5
                                         ..
Fantastic experience every time           1
It's incredible, I’m hooked               1
Too many issues lately                    1
Dissatisfied with features                1
Great service but needs more features.    1
Name: count, Length: 1254, dtype: int64


#### Deleting Rows with Non-English Data

In [75]:
def is_english_subscription(text):
    try:
        lang, confidence = langid.classify(str(text))
        return lang == "en" and confidence > 0.7
    except:
        return False

In [76]:
DetectorFactory.seed = 0

def is_english_feedback(text):
    try:
        return detect(str(text)) == "en"
    except:
        return False

In [77]:
df_clean['subscription_type_en'] = df_clean['subscription_type'].apply(is_english_subscription)
df_clean['feedback_en'] = df_clean['customer_feedback'].apply(is_english_feedback)

#### Verifying the Classification

In [79]:
print("Non-English subscription types:")
print(df_clean[~df_clean['subscription_type_en']]['subscription_type'].value_counts())

print("Non-English feedback samples:")
print(df_clean[~df_clean['feedback_en']]['customer_feedback'])

Non-English subscription types:
subscription_type
国产    1
Name: count, dtype: int64
Non-English feedback samples:
6                Excellent value for money!
9                   Could use more updates.
11      Decent service, needs improvements.
15           Very good overall, keep it up!
25                 I feel it lacks variety.
                       ...                 
1326                     Excellent service!
1331      I'm leaving for something better.
1336                    I might cancel soon
1364             Not good, will likely quit
1369                         It's just okay
Name: customer_feedback, Length: 123, dtype: object


In [81]:
df_clean[~df_clean['feedback_en']]['customer_feedback'].to_list()

['Excellent value for money!',
 'Could use more updates.',
 'Decent service, needs improvements.',
 'Very good overall, keep it up!',
 'I feel it lacks variety.',
 'Always satisfied, keep it going!',
 "Best investment I've made!",
 'I love it, never a problem!',
 'Best decision I ever made!',
 'Best decision I ever made!',
 'Not meeting my needs; unhappy.',
 'Enjoyable experience overall!',
 'Best platform ever!',
 'I’m leaving for better deals',
 "Service doesn't meet my needs",
 'Best decision I made!',
 'Fantastic user experience',
 'Enjoyable experience overall',
 'Decent but not excellent',
 'Very reliable and enjoyable',
 "Best decision I've made for entertainment",
 'Fantastic service, no complaints',
 'I am leaving for better options',
 'Not impressed, likely leaving.',
 'Best decision I made subscribing!',
 "Best experience I've had!",
 "Best app I've ever used!",
 'Not engaging enough for me.',
 'I need to cancel soon.',
 "Best investment I've made!",
 'Excellent support and 

Since customer_feedbacks are wrongly classified as non-english,we can safely keep them

#### Removing Non-English Rows for subscription_type

In [82]:
df_clean = df_clean[df_clean['subscription_type_en']].reset_index(drop=True)
print(f"Dataset after removing non-English rows: {len(df_clean)} rows")

Dataset after removing non-English rows: 1372 rows


## Final Dataset

In [83]:
df_clean = df_clean.drop(columns=["subscription_type_en", "feedback_en"])

In [84]:
df_clean

,customer_id,age,gender,subscription_type,tenure_months,monthly_spend,support_tickets,avg_session_time,customer_feedback,churn
0,85714623.0,34.0,M,Standard,12.0,45.50,3.0,30.5,Great service but needs more features.,0.0
1,29547381.0,28.0,F,Premium,24.0,150.00,1.0,75.2,Love the content offerings!,0.0
2,33416294.0,45.0,M,Standard,9.0,50.00,6.0,35.0,"Not bad, but slow response times.",1.0
3,19486372.0,55.0,F,Premium,36.0,180.00,0.0,90.0,Best service I've used so far!,0.0
4,87126314.0,22.0,M,Standard,2.0,30.00,8.0,20.0,I might consider leaving soon.,1.0
...,...,...,...,...,...,...,...,...,...,...
1367,86713409.0,22.0,F,Standard,1.0,12.50,15.0,10.5,Not happy with the service,1.0
1368,48152036.0,32.0,M,Basic,9.0,27.30,7.0,24.0,It's just okay,1.0
1369,39164782.0,26.0,F,Premium,3.0,135.00,1.0,100.0,Very nice experience overall,0.0
1370,29908417.0,48.0,M,Standard,37.0,99.99,2.0,85.0,Service has been reliable,0.0


## Exporting Dataset

In [85]:
df_clean.to_csv("clean_customer_churn_dataset.csv", index=False)